In [ ]:
# Logistic regression : can we predict whether or not male participants have Diabetes based on their year of birth, height, weight, smoking status
# Input data : participant-data.pkl / participant-data.csv etc. created in script 1-tre-test-prep 

In [ ]:
# import libraries 

# please try to to import libraries stored in another jupyter notebook
# this notebook is called 'my_libraries'
# and is imported as follows

import import_ipynb
from my_libraries import *

In [ ]:
# if importing via a notebook does not work, please uncomment the below

#import io, os, sys, types
#from IPython import get_ipython
#from nbformat import read
#from IPython.core.interactiveshell import InteractiveShell
#import numpy as np
#import pandas as pd
#from datetime import date
#from scipy import stats
#from scipy.stats import norm
#import seaborn as sns
#import matplotlib
#import matplotlib.pyplot as plt
#import pickle
#from sklearn.linear_model import LogisticRegression
#from sklearn.model_selection import StratifiedShuffleSplit
#from sklearn.metrics import confusion_matrix, accuracy_score
#from sklearn.preprocessing import StandardScaler
#from sklearn.preprocessing import MinMaxScaler
#from sklearn.model_selection import train_test_split
#from sklearn import metrics
#import sklearn.metrics as metrics
#from sklearn.metrics import classification_report
#sns.set_palette("BuPu")

In [ ]:
# set directories 

os.chdir('/Users/bethanyisaacs/Documents/git-new/tre-demo-epi-test/DemoEpidemiologyScenario/')

indir = os.getcwd()+'/OFH-data'
outdir = os.getcwd()+'/_outputs'

# read in from pickle

f = open(outdir+"/participant-data.pkl",'rb') 
dat = pickle.load(f)[0]

In [ ]:
# # # Data Preparation 

# we want to see if smoking status, sex as male, year of birth, height and or weight can predict whether a participant is diabetic or not
# some variables we want to use as predictors are cateorigcal, and we only actually want one level, so dummy code those levels 

dat['ICD code'].fillna(0, inplace=True) # clean the data : some missing values are present in the ICD code column - we might not use it but just incase, fillna
dat['smoking 0'] = (dat['smoking'] == 'Yes, on most or all days').astype(int) # dummy code smoking status == 1 if the response == 'Yes, on most or all days' else == 0
dat['male 0'] = (dat['sex'] == 'Male').astype(int) # dummy code male == 1 if the response == 'Male' , else == 0
dat_sub = dat[['male 0', 'year of birth', 'height', 'weight', 'diabetic status', 'smoking 0']] # subset the dataset to include only variables we want for our analysis

the subsampled dataset now has 6 variables
we have converted all relevant categorical variables in to dummy codes
- Male 0 (1=yes, 0 = no)
- Smoking 0 (1 = yes, 0 = no)
- year of birth (year)
- height (cm)
- weight (kg)
- diabetic status (1 = diabeteic, 0 = non diabetic)

In [ ]:
# # # Head of the subsampled dataset that will be used for analysis 

dat_sub.head()

In [ ]:
# # # Visualising the Target Variable 
 
dat_sub['diabetic status'].value_counts(normalize=True)*100 # the dataset is skewed - 87% of participants do not have diabetes 
sns.countplot( x = 'diabetic status', data = dat_sub)

In [ ]:
# # # Pairplot to visualise relationships and distributions amongst variables 

sns.pairplot(dat_sub) 

In [ ]:
# # # Heatmap for correlations amongst variables

sns.heatmap(dat_sub.corr(), annot=True, fmt='.2f') 

In [ ]:
# # # Prepare Data

x = np.asarray(dat_sub.drop(['diabetic status'], axis=1)) # split data in to features (x) 
y = np.asarray(dat_sub['diabetic status']) # split the data in to targets (y)
x = StandardScaler().fit(x).transform(x) # normalize conintious numeric variables
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, shuffle=True, random_state = 4) # split the dataframe in to test and train sets 

In [ ]:
# # # Run Logistic Regresskion

model = LogisticRegression(random_state=12, C = 0.1, penalty = 'l2', solver='liblinear', class_weight="balanced") # initiate logistic regression 
model.fit(X_train, y_train) # fit model  
y_pred = model.predict(X_test) # get predictions 

In [ ]:
# # # Model accuracy 

print('Training accuracy : %.2f' % (model.score(X_train, y_train)*100).round(2)) # evaluate training models 
print('Test accuracy : %.2f' % (model.score(X_test, y_test)*100).round(2)) # evaluate testing models 
print('Predictive accuracy : %.2f' % (metrics.accuracy_score(y_pred, y_test)*100).round(2)) # get accuracy of predictive model (proportion of correct predictions over total predictions)

In [ ]:
# # # Classification Report
 
print(classification_report(y_test, y_pred))

In [ ]:
# # # Heatmap : Confusion Matrix

cm  = confusion_matrix(y_test, y_pred) # create confusion matrix to visualise actial and predicted values 
cm_setup = pd.DataFrame(data=cm, columns = [ 'predict : 0', 'predict : 1' ], index = [ 'actual : 0', 'actual : 1'])
sns.heatmap(cm_setup, annot = True, fmt='d')

In [ ]:
# # # Totals : Confusion Matrix

tn, fp, fn, tp = metrics.confusion_matrix(y_test, y_pred).ravel() # print the values in terms of true/false negative/positives
print("True Negatives: ",tn)
print("False Positives: ",fp)
print("False Negatives: ",fn)
print("True Positives: ",tp)

In [ ]:
# # # ROC and AUC

y_pred_proba = model.predict_proba(X_test)[:,1] # infer probabilities 
fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba) # calculate ROC
auc = metrics.roc_auc_score(y_test, y_pred_proba) # calculate AUC

plt.plot(fpr,tpr,label="AUC="+str(auc), color='red') # plot false positive rate and true positive rates, add AUC score to table
plt.ylabel('True Positive Rate') # add labels
plt.xlabel('False Positive Rate')
plt.legend(loc=4) # add legend
plt.show()

In [ ]:
# # # Feature importance 

coeff = list(model.coef_[0]) # get coefficients

features = pd.DataFrame() # initiate dataframe

features['Features'] = list(dat_sub.loc[:, dat_sub.columns !='diabetic status'].columns) # insert gefeatures as index
features['importance'] = coeff # insert coefficients per feature

features.sort_values(by=['importance'], ascending=True, inplace=True) # sort by importance
features['positive'] = features['importance'] > 0 # label positive or negative
features.set_index('Features', inplace=True) # set features as index

features.importance.plot(kind='barh', figsize=(11, 6),color = features.positive.map({True: 'blue', False: 'red'})) # bar plot 
f = plt.xlabel('Importance') # title